## Wczytanie danych i bibliotek

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets

In [ ]:
file = "ks_projects_201801.csv"
filepath = os.path.join(os.path.abspath(""), file)

In [ ]:
df = pd.read_csv(filepath)


## Wstępne sprawdzenie danych

In [ ]:
# Usunięcie wartości NaN
df = df.dropna()

# Ujednolicenie kolumn z datami, przekształcenie na format datetime
df['deadline_new'] = pd.to_datetime(df['deadline']).dt.date
df['launched_new'] = pd.to_datetime(df['launched']).dt.date

# Obliczenie czasu trwania projektów, oczyszczenie danych - przekształcenie na typ int
df['duration'] = df['deadline_new'] - df['launched_new']
df['duration'] = df['duration'].map(lambda x: int(str(x).split(" ")[0]))

# Wyodrębnienie dwóch stanów projektów: successful = 1, pozostałe = 0
df["state_new"] = df["state"].map(lambda x: 1 if x == "successful" else 0 )


df.sample(5)

In [ ]:
df.columns

In [ ]:
# Usunięcie niepotrzebnych kolumn
to_drop = ["deadline",
           "launched",
           "state",
           "usd pledged"]

df.drop(to_drop, inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
# Sprawdzenie, czy projekty się powtarzają
print(df["ID"].duplicated().any())

## Sprawdzenie czy powodzenie projektu zależy od kraju

In [ ]:
# Sprawdzenie ile było projektów w poszczególnych krajach
project_country = dict(df["country"].value_counts())

project_country

In [ ]:
# Połączenie Luksemburga, Belgii i Holandii w Beneluks

df["country"] = df["country"].replace(["LU", "BE"],"NL")

project_country = dict(df["country"].value_counts())
project_country

In [ ]:
# Odrzucenie krajów, w których liczba projektów jest poniżej 1000

df = df[(df["country"] != "SG") 
        & (df["country"] != "JP") 
        & (df["country"] != "AT") 
        & (df["country"] != "HK")
        & (df["country"] != "NO")
        & (df["country"] != "CH")
        & (df["country"] != "IE")]
df["country"].unique()

In [ ]:
# Odfiltrowanie projektów, które odniosły sukces w poszczególnych krajach

success_filter = df[df["state_new"] == 1]
group_success_filter = success_filter.groupby(by=["country"])
sum_success = dict(group_success_filter["state_new"].sum())

print(sum_success)

In [ ]:
# Procent projektów, które się udały

perc_success = {k: round(((sum_success[k] / project_country[k]) * 100), 1) for k, v in sum_success.items()}

print(perc_success)

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od kraju

perc_country_list = []
country_list = []
for k, v in perc_success.items():
    perc_country_list.append(v)
    country_list.append(k)
        
# print(perc_country_list)
# print(country_list)

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(range(len(perc_country_list)), perc_country_list, align='center')
plt.xticks(range(len(perc_country_list)), country_list, rotation='vertical')
plt.show()

## Sprawdzenie czy powodzenie projektu zależy od jego daty złożenia lub zakończenia (miesiąc)

In [ ]:
# Wyodrębnienie roku i miesiąca z danych 

df['year_launched'] = pd.DatetimeIndex(df['launched_new']).year
df['month_launched'] = pd.DatetimeIndex(df['launched_new']).month
df['year_deadline'] = pd.DatetimeIndex(df['deadline_new']).year
df['month_deadline'] = pd.DatetimeIndex(df['deadline_new']).month
df.head()

In [ ]:
# Sprawdzenie ile projektów powstało w poszczególnych latach

df["year_launched"].value_counts()

In [ ]:
# Usunięcie rekordów z 1970 roku

df = df[df["year_launched"] != 1970]
df["year_launched"].unique()

In [ ]:
# Ile było projektów w poszczególnych krajach w danym miesiącu

all_month_grouped = df.groupby(by=["country", "month_launched"])
all_month = dict(all_month_grouped["state_new"].count())
# all_month

In [ ]:
# Ile było projektów, które zostały sfinansowane w poszczególnych krajach w danym miesiącu

filter_month = df[df["state_new"] == 1]
month_grouped = filter_month.groupby(by=["country", "month_launched"])
success_month = dict(month_grouped["state_new"].count())
#success_month

In [ ]:
# Procent projektów, które zostały sfinansowane w poszczególnych krajach w zależności od miesiąca rozpoczęcia projektu

perc_month = {k: round(((success_month[k] / all_month[k]) * 100), 1) for k, v in success_month.items()}

# perc_month

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od miesiąca rozpoczęcia

perc_month_list = []
month_list = []
for k, v in perc_month.items():
    if "GB" in k:
#         print(k, v)
        perc_month_list.append(v)
        month_list.append(k)
        
# print(perc_month_list)
# print(month_list)
# print(df["country"].unique())

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(range(len(perc_month_list)), perc_month_list, align='center')
plt.xticks(range(len(perc_month_list)), month_list, rotation='vertical')
plt.show()

#### W dużej części państw odsetek projektów zakończonych powodzeniem nie zależy od miesiąca, w którym projekty były składane, więc nie bieżemy tej zależności pod uwagę w budowaniu aplikacji.

## Sprawdzenie czy powodzenie projektu zależy od kategorii w której został złożony

In [ ]:
df["main_category"].unique()

In [ ]:
# Ile było projektów w poszczególnych krajach w danej kategorii

all_mcategory_grouped = df.groupby(by=["country", "main_category"])
all_mcategory = dict(all_mcategory_grouped["state_new"].count())
# all_mcategory


In [ ]:
# Ile było projektów, które zostały sfinansowane w poszczególnych krajach w danej kategorii

filter_mcategory = df[df["state_new"] == 1]
mcategory_grouped = filter_mcategory.groupby(by=["country", "main_category"])
success_mcategory = dict(mcategory_grouped["state_new"].count())
# success_mcategory

In [ ]:
# Procent projektów, które zostały sfinansowane w poszczególnych krajach w zależności od kategorii

perc_mcategory = {k: round(((success_mcategory[k] / all_mcategory[k]) * 100), 1) for k, v in success_mcategory.items()}

# perc_mcategory

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od kategorii, w której projekt został złożony

value_list = []
mcat_list = []
for k, v in perc_mcategory.items():
    if "US" in k:
#         print(k, v)
        value_list.append(v)
        mcat_list.append(k)
        
# print(value_list)
# print(mcat_list)

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(range(len(value_list)), value_list, align='center')
plt.xticks(range(len(value_list)), mcat_list, rotation='vertical')
plt.show()

#### We wszystkich państwach widać wyraźną zależność powodzenia projektu od tego w jakiej kategorii został złożony, więc kategoria będzie jednym z parametrów analizowanych w aplikacji.

## Sprawdzenie czy powodzenie projektu zależy od podkategorii w której został złożony

In [ ]:
# df["category"].unique()

In [ ]:
# Ile było projektów w poszczególnych krajach w danej podkategorii

all_category_grouped = df.groupby(by=["country", "main_category", "category"])
all_category = dict(all_category_grouped["state_new"].count())
# all_category

In [ ]:
# Ile było projektów, które zostały sfinansowane w poszczególnych krajach w danej podkategorii

filter_category = df[df["state_new"] == 1]
category_grouped = filter_category.groupby(by=["country", "main_category", "category"])
success_category = dict(category_grouped["state_new"].count())
# success_category

In [ ]:
# Procent projektów, które zostały sfinansowane w poszczególnych krajach w zależności od podkategorii

perc_category = {k: round(((success_category[k] / all_category[k]) * 100), 1) for k, v in success_category.items()}

# perc_category

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od podkategorii, w której projekt został złożony

per_cat_list = []
cat_list = []
for k, v in perc_category.items():
    if "IT" in k:
#         print(k, v)
        per_cat_list.append(v)
        cat_list.append(k)
        
# print(per_cat_list)
# print(cat_list)

In [ ]:
plt.figure(figsize=(25, 10))
plt.bar(range(len(per_cat_list)), per_cat_list, align='center')
plt.xticks(range(len(per_cat_list)), cat_list, rotation='vertical')
plt.show()

#### We wszystkich państwach widać wyraźną zależność powodzenia projektu od tego w jakiej podkategorii został złożony, więc podkategoria będzie jednym z parametrów analizowanych w aplikacji.

## Opracowanie funkcji pozwalającej na sprawdzenie kilku czynników jednocześnie

In [ ]:
# Funkcja pozwalająca na sprawdzenie wszystkich czynników jednocześnie 

def success_rate(groupby, column, country):
    
    for el in groupby:
        
        # Liczba projektów w poszczególnych krajach w danej grupie czynników
        
        all_grouped = df.groupby(by=el)
        all_count = dict(all_grouped[column].count())
        
        # Liczba projektów, które zostały sfinansowane w poszczególnych krajach w danej grupie czynników

        filter_success = df[df[column] == 1]
        factor_grouped = filter_success.groupby(by=el)
        success_factor = dict(factor_grouped[column].count())
        
        # Procent projektów, które zostały sfinansowane w poszczególnych krajach w zależności od czynnika

        perc_factor = {k: round(((success_factor[k] / all_count[k]) * 100), 1) for k, v in success_factor.items()}

        # Wizualizacja odsetka sfinansowanych projektów w zależności od miesiąca rozpoczęcia

        perc_factor_list = []
        factor_list = []
        for key, value in perc_factor.items():
            if country in key:
                perc_factor_list.append(value)
                factor_list.append(key)
                
        plt.figure(figsize=(25, 5))
        plt.title(el[-1])
        plt.bar(range(len(perc_factor_list)), perc_factor_list, align='center')
        plt.xticks(range(len(perc_factor_list)), factor_list, rotation='vertical')
        plt.show()

In [ ]:
list_groupby = [["country", "month_launched"], ["country", "main_category"], ["country", "main_category", "category"]]
column_count = "state_new"
country_list = ['GB', 'US', 'CA', 'AU', 'NO', 'IT', 'DE', 'IE', 'MX', 'ES', 'SE','FR', 'NL', 'NZ', 'CH', 'AT', 'DK', 'BE', 'HK', 'SG']

success_rate(list_groupby, column_count, "IT")

## Sprawdzenie czy powodzenie projektu zależy od czasu trwania zbiórki

In [ ]:
df["duration"].hist(bins=10, figsize=(15, 5))

In [ ]:
df["duration"].quantile([0.25, 0.50, 0.75])

In [ ]:
df["duration"].value_counts()

In [ ]:
# Określenie liczby projektów z danym czasem zbiórki (co 10 dni)

set_1 = df[df["duration"] < 10]["duration"].count()
print(f"Poniżej 10 dni: {set_1}")
set_2 = df[(df["duration"] >= 10) & (df["duration"] < 20)]["duration"].count()
print(f"Od 10 do 20 dni: {set_2}")
set_3 = df[(df["duration"] >= 20) & (df["duration"] < 30)]["duration"].count()
print(f"Od 20 do 30 dni: {set_3}")
set_4 = df[(df["duration"] >= 30) & (df["duration"] < 40)]["duration"].count()
print(f"Od 30 do 40 dni: {set_4}")
set_5 = df[(df["duration"] >= 40) & (df["duration"] < 50)]["duration"].count()
print(f"Od 40 do 50 dni: {set_5}")
set_6 = df[(df["duration"] >= 50) & (df["duration"] < 60)]["duration"].count()
print(f"Od 50 do 60 dni: {set_6}")
set_7 = df[(df["duration"] >= 60) & (df["duration"] < 70)]["duration"].count()
print(f"Od 60 do 70 dni: {set_7}")
set_8 = df[(df["duration"] >= 70) & (df["duration"] < 80)]["duration"].count()
print(f"Od 70 do 80 dni: {set_8}")
set_9 = df[(df["duration"] >= 80) & (df["duration"] < 90)]["duration"].count()
print(f"Od 80 do 90 dni: {set_9}")
set_10 = df[(df["duration"] >= 90) & (df["duration"] < 100)]["duration"].count()
print(f"Od 90 do 100 dni: {set_10}")

In [ ]:
# Określenie liczby projektów zakończonych sukcesem z danym czasem zbiórki (co 10 dni)

set_1s = df[(df["state_new"] == 1) & (df["duration"] < 10)]["duration"].count()
print(f"Poniżej 10 dni: {set_1s}")
set_2s = df[(df["state_new"] == 1) & (df["duration"] >= 10) & (df["duration"] < 20)]["duration"].count()
print(f"Od 10 do 20 dni: {set_2s}")
set_3s = df[(df["state_new"] == 1) & (df["duration"] >= 20) & (df["duration"] < 30)]["duration"].count()
print(f"Od 20 do 30 dni: {set_3s}")
set_4s = df[(df["state_new"] == 1) & (df["duration"] >= 30) & (df["duration"] < 40)]["duration"].count()
print(f"Od 30 do 40 dni: {set_4s}")
set_5s = df[(df["state_new"] == 1) & (df["duration"] >= 40) & (df["duration"] < 50)]["duration"].count()
print(f"Od 40 do 50 dni: {set_5s}")
set_6s = df[(df["state_new"] == 1) & (df["duration"] >= 50) & (df["duration"] < 60)]["duration"].count()
print(f"Od 50 do 60 dni: {set_6s}")
set_7s = df[(df["state_new"] == 1) & (df["duration"] >= 60) & (df["duration"] < 70)]["duration"].count()
print(f"Od 60 do 70 dni: {set_7s}")
set_8s = df[(df["state_new"] == 1) & (df["duration"] >= 70) & (df["duration"] < 80)]["duration"].count()
print(f"Od 70 do 80 dni: {set_8s}")
set_9s = df[(df["state_new"] == 1) & (df["duration"] >= 80) & (df["duration"] < 90)]["duration"].count()
print(f"Od 80 do 90 dni: {set_9s}")
set_10s = df[(df["state_new"] == 1) & (df["duration"] >= 90) & (df["duration"] < 100)]["duration"].count()
print(f"Od 90 do 100 dni: {set_10s}")

In [ ]:
# Odsetek projektów zakończonych sukcesem w zależności od długości zbiórki

set_1p = 100 * set_1s / set_1
print(f"Poniżej 10 dni: {set_1p:.1f} %")
set_2p = 100 * set_2s / set_2
print(f"Od 10 do 20 dni: {set_2p:.1f} %")
set_3p = 100 * set_3s / set_3
print(f"Od 20 do 30 dni: {set_3p:.1f} %")
set_4p = 100 * set_4s / set_4
print(f"Od 30 do 40 dni: {set_4p:.1f} %")
set_5p = 100 * set_5s / set_5
print(f"Od 40 do 50 dni: {set_5p:.1f} %")
set_6p = 100 * set_6s / set_6
print(f"Od 50 do 60 dni: {set_6p:.1f} %")
set_7p = 100 * set_7s / set_7
print(f"Od 60 do 70 dni: {set_7p:.1f} %")
set_8p = 100 * set_8s / set_8
print(f"Od 70 do 80 dni: {set_8p:.1f} %")
set_9p = 100 * set_9s / set_9
print(f"Od 80 do 90 dni: {set_9p:.1f} %")
set_10p = 100 * set_10s / set_10
print(f"Od 90 do 100 dni: {set_10p:.1f} %")

In [ ]:
set_list = [set_1p, set_2p, set_3p, set_4p, set_5p, set_6p, set_7p, set_8p, set_9p, set_10p]

plt.figure(figsize=(10, 5))
plt.bar(range(len(set_list)), set_list, align='center')
plt.xticks(range(len(set_list)), range(10, 101, 10), rotation='vertical')
plt.show()

#### Możliwe jest określenie przedziałów czasowych, które mogłyby zwiększyć prawdopodobieństwo zakończenia zbiórki sukcesem

## Sprawdzenie czy powodzenie projektu zależy od kwoty zbiórki

In [ ]:
df["usd_goal_real"].describe()

In [ ]:
df["usd_pledged_real"].describe()

In [ ]:
quant = dict(df["usd_goal_real"].quantile([0.2, 0.4, 0.6, 0.8]))

In [ ]:
q2 = 100 * df[(df["usd_goal_real"] < quant[0.2]) & (df["state_new"] == 1)]["ID"].count() / df[(df["usd_goal_real"] < quant[0.2])]["ID"].count() 
q4 = 100 * df[(df["usd_goal_real"] >= quant[0.2]) & (df["usd_goal_real"] < quant[0.4]) & (df["state_new"] == 1)]["ID"].count() / df[(df["usd_goal_real"] >= quant[0.2]) & (df["usd_goal_real"] < quant[0.4])]["ID"].count()
q6 = 100 * df[(df["usd_goal_real"] >= quant[0.4]) & (df["usd_goal_real"] < quant[0.6]) & (df["state_new"] == 1)]["ID"].count() / df[(df["usd_goal_real"] >= quant[0.4]) & (df["usd_goal_real"] < quant[0.6])]["ID"].count()
q8 = 100 * df[(df["usd_goal_real"] >= quant[0.6]) & (df["usd_goal_real"] < quant[0.8]) & (df["state_new"] == 1)]["ID"].count() / df[(df["usd_goal_real"] >= quant[0.6]) & (df["usd_goal_real"] < quant[0.8])]["ID"].count()
q10 = 100 * df[(df["usd_goal_real"] >= quant[0.8]) & (df["state_new"] == 1)]["ID"].count() / df[(df["usd_goal_real"] >= quant[0.8])]["ID"].count() 

list_q = [q2, q4, q6, q8, q10]

plt.figure(figsize=(10, 5))
plt.bar(range(len(list_q)), list_q, align='center')
plt.xticks(rotation='horizontal')
plt.show()